# Using PineconeLocalIndex for Routes

Pinecone Local is an in-memory Pinecone Database emulator available as a Docker image.

It's useful for running tests using the Pinecone local server. Your data will not leave your system, which is also helpful if you want to do testing locally before committing to a Pinecone account.

## Limitations
Pinecone Local has the following limitations:

- Pinecone Local is available only as a Docker image.
- Pinecone Local is an in-memory emulator and is not suitable for production. Records loaded into Pinecone Local do not persist after it is stopped.
- Pinecone Local does not authenticate client requests. API keys are ignored.
- Max number of records per index: 100,000.

## Getting Started
Make sure [Docker](https://docs.docker.com/get-docker/) is installed and running on your local machine.

### Download the latest pinecone-local Docker image:


Download the latest pinecone-local Docker image:

```bash
docker pull ghcr.io/pinecone-io/pinecone-local:latest
```

### Start Pinecone Local:

```bash
docker run -d \
--name pinecone-local \
-e PORT=5080 \
-e PINECONE_HOST=localhost \
-p 5080-6000:5080-6000 \
--platform linux/amd64 \
ghcr.io/pinecone-io/pinecone-local:latest
```



In [16]:
!pip install -qU "semantic-router[pinecone]>=0.1.5"

In [1]:
from semantic_router import Route

# we could use this as a guide for our chatbot to avoid political conversations
politics = Route(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the presidentdon't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
    ],
)

# this could be used as an indicator to our chatbot to switch to a more
# conversational prompt
chitchat = Route(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
    ],
)

# we place both of our decisions together into single list
routes = [politics, chitchat]

c:\Users\awesome\anaconda3\envs\py12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
routes

[Route(name='politics', utterances=["isn't politics the best thing ever", "why don't you tell me about your political opinions", "don't you just love the presidentdon't you just hate the president", "they're going to destroy this country!", 'they will save the country!'], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={}),
 Route(name='chitchat', utterances=["how's the weather today?", 'how are things going?', 'lovely weather today', 'the weather is horrendous', "let's go to the chippy"], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={})]

In [3]:
import os
from getpass import getpass
from semantic_router.encoders import OpenAIEncoder

# get at platform.openai.com
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY") or getpass(
    "Enter OpenAI API key: "
)
encoder = OpenAIEncoder(
    name="text-embedding-3-large", score_threshold=0.5, dimensions=1536
)

For Pinecone Local, you can pass the API key as `pclocal`.

In [4]:
import os
from semantic_router.index.pinecone import PineconeIndex

os.environ["PINECONE_API_KEY"] = os.environ.get("PINECONE_API_KEY") or getpass(
    "Enter Pinecone API key: "
)

# Pass the pinecone local hosted url as base url
# os.environ["PINECONE_API_BASE_URL"] = "https://api.pinecone.io"
os.environ["PINECONE_API_BASE_URL"] = "http://localhost:5080"

index = PineconeIndex(
    index_name="route-test", dimensions=1536, base_url="http://localhost:5080"
)

2025-02-03 15:40:01 - semantic_router.utils.logger - WARNING - pinecone.py:169 - __init__() - Default region changed from us-west-2 to us-east-1 in v0.1.0.dev6
2025-02-03 15:40:01 - pinecone_plugin_interface.logging - INFO - discover_namespace_packages.py:12 - discover_subpackages() - Discovering subpackages in _NamespacePath(['c:\\Users\\awesome\\anaconda3\\envs\\py12\\Lib\\site-packages\\pinecone_plugins'])
2025-02-03 15:40:01 - pinecone_plugin_interface.logging - INFO - discover_plugins.py:9 - discover_plugins() - Looking for plugins in pinecone_plugins.inference
2025-02-03 15:40:01 - pinecone_plugin_interface.logging - INFO - installation.py:10 - install_plugins() - Installing plugin inference into Pinecone
2025-02-03 15:40:01 - semantic_router.utils.logger - ERROR - pinecone.py:233 - _init_index() - index_host exists-pinecone:localhost:5081
2025-02-03 15:40:01 - semantic_router.utils.logger - ERROR - pinecone.py:234 - _init_index() - base_url exists-pinecone:http://localhost:5080


In [8]:
from semantic_router.routers import SemanticRouter

router = SemanticRouter(
    encoder=encoder,
    routes=routes,
    index=index,
    auto_sync="local",
)

2025-02-03 15:41:01 - semantic_router.utils.logger - ERROR - pinecone.py:233 - _init_index() - index_host exists-pinecone:localhost:5081
2025-02-03 15:41:01 - semantic_router.utils.logger - ERROR - pinecone.py:234 - _init_index() - base_url exists-pinecone:http://localhost:5080
2025-02-03 15:41:01 - semantic_router.utils.logger - ERROR - pinecone.py:251 - _init_index() - index_host exists:http://localhost:5081
2025-02-03 15:41:01 - semantic_router.utils.logger - ERROR - pinecone.py:252 - _init_index() - index exists:<pinecone.data.index.Index object at 0x000002C6CAA8FB90>


We can check our route layer and index information.

In [9]:
index.index_host

'http://localhost:5081'

In [10]:
router.index.index_host

'http://localhost:5081'

In [11]:
router.index.client.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 1536,
              'host': 'localhost:5081',
              'metric': 'dotproduct',
              'name': 'route-test',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [12]:
router.index.is_ready()

True

In [13]:
router.is_synced()

True

In [14]:
router.get_utterance_diff()

['  chitchat: how are things going?',
 "  chitchat: how's the weather today?",
 "  chitchat: let's go to the chippy",
 '  chitchat: lovely weather today',
 '  chitchat: the weather is horrendous',
 "  politics: don't you just love the presidentdon't you just hate the president",
 "  politics: isn't politics the best thing ever",
 '  politics: they will save the country!',
 "  politics: they're going to destroy this country!",
 "  politics: why don't you tell me about your political opinions"]

In [15]:
router("don't you love politics?")

2025-02-03 15:41:08 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:41:08 - semantic_router.utils.logger - ERROR - pinecone.py:575 - query() - retrying query with vector as str


RouteChoice(name='politics', function_call=None, similarity_score=None)

In [17]:
router.list_route_names()

['politics', 'chitchat']

In [18]:
router("how are things going?")

2025-02-03 15:41:17 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:41:17 - semantic_router.utils.logger - ERROR - pinecone.py:575 - query() - retrying query with vector as str


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

In [19]:
len(router.index)

10

We can also view all of the records for a given route:

In [20]:
router.index._get_route_ids(route_name="politics")

['politics#64069085d9d6e98e5a80915f69fabe82bac6c742f801bc305c5001dce88f0d19',
 'politics#af8b76111f260cf44fb34f04fcf82927dcbe08e8f47c30f4d571379c1512fac8',
 'politics#d1bb40236c3d95b9c695bfa86b314b6da4eb87e136699563fccae47fccea23e2',
 'politics#ed0f3dd7bd5dea12e55b1953bcd2c562a5ab19f501f6d5ff8c8927652c3904b8',
 'politics#fc6d15f9e6075e6de82b3fbef6722b64353e4eadc8d663b7312a4ed60c43e6f6']

And query:

In [21]:
router("don't you love politics?").name

2025-02-03 15:41:25 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:41:25 - semantic_router.utils.logger - ERROR - pinecone.py:575 - query() - retrying query with vector as str


'politics'

In [22]:
router("how's the weather today?").name

2025-02-03 15:19:14 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:19:14 - semantic_router.utils.logger - ERROR - pinecone.py:572 - query() - retrying query with vector as str


'chitchat'

In [22]:
router("I'm interested in learning about llama 2").name

2025-02-03 15:41:32 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:41:32 - semantic_router.utils.logger - ERROR - pinecone.py:575 - query() - retrying query with vector as str


We can delete or update routes.

In [24]:
len(router.index)

10

## Deleting a Route from the Semantic Router
In this section, we demonstrate how to delete a specific route from the `SemanticRouter` instance. This is useful when you want to remove a route that is no longer needed or to update the routing logic dynamically.


In [23]:
import time

router.delete(route_name="politics")
time.sleep(1)
len(router.index)

2025-02-03 15:41:39 - semantic_router.utils.logger - WARNING - pinecone.py:611 - _read_config() - Configuration for sr_lock parameter not found in index.
2025-02-03 15:41:39 - semantic_router.utils.logger - INFO - pinecone.py:477 - delete() - index is not None, deleting...
2025-02-03 15:41:39 - semantic_router.utils.logger - INFO - pinecone.py:491 - delete() - Deleted 5 vectors from index route-test.


5

In [24]:
router("don't you love politics??").name

2025-02-03 15:41:49 - httpx - INFO - _client.py:1013 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-02-03 15:41:49 - semantic_router.utils.logger - ERROR - pinecone.py:575 - query() - retrying query with vector as str


In [25]:
router.index.get_routes()

[Route(name='chitchat', utterances=['how are things going?', "how's the weather today?", 'the weather is horrendous', 'lovely weather today', "let's go to the chippy"], description=None, function_schemas=None, llm=None, score_threshold=None, metadata={})]

## Asynchronous Route Query with Semantic Router

In this section, we explore how to perform an asynchronous query using the `SemanticRouter`. This approach is beneficial when you want to handle multiple queries concurrently without blocking the execution of your program.

In [26]:
await router.acall("how are things going?")

2025-02-03 15:42:01 - httpx - INFO - _client.py:1729 - _send_single_request() - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


RouteChoice(name='chitchat', function_call=None, similarity_score=None)

In [27]:
await router.index._async_list_indexes()

{'indexes': [{'name': 'route-test',
   'dimension': 1536,
   'metric': 'dotproduct',
   'host': 'localhost:5081',
   'deletion_protection': 'disabled',
   'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
   'status': {'ready': True, 'state': 'Ready'}}]}

### Stop and remove Pinecone Local

To stop and remove the resources for Pinecone Local, run the following command:

```bash
docker compose down
docker stop pinecone-local
docker rm pinecone-local
```